# Kaggle survey

Info about the competition: https://www.kaggle.com/competitions/kaggle-survey-2022/overview


In [1]:
#!pip install ipyvizzu
#!pip install ipyvizzu-story

In [2]:
import pandas as pd
import numpy as np

from ipyvizzu import Chart, Data, Config, Style
from ipyvizzustory import Story, Slide, Step

#Create data object, read csv to data frame and add data frame to data object.
data = Data()

#Only use columns you'll need for your story - should be updated to match the story content
col_list = ['Q2','Q3','Q5','Q29','Q23']

df = pd.read_csv("../data/Kaggle/kaggle_survey_2022_responses.csv", usecols=col_list)
pd.set_option('display.max_columns', None)



#Add count column
df['Count'] = 1

df.head()

,Q2,Q3,Q5,Q23,Q29,Count
0,What is your age (# years)?,What is your gender? - Selected Choice,"Are you currently a student? (high school, uni...",Select the title most similar to your current ...,What is your current yearly compensation (appr...,1
1,30-34,Man,No,NaN,NaN,1
2,30-34,Man,No,NaN,NaN,1
3,18-21,Man,Yes,NaN,NaN,1
4,55-59,Man,No,Data Scientist,"25,000-29,999",1


In [3]:
#Remove row with question texts
df = df.drop(0)

#List unique answers for Q2
df.Q2.unique()

array(['30-34', '18-21', '55-59', '45-49', '70+', '22-24', '35-39',
       '40-44', '50-54', '25-29', '60-69'], dtype=object)

In [4]:
#Add new column categorizing respondents with ages above and below 30
df.insert(2,'below_30',np.where((df.Q2 == '18-21') | (df.Q2 == '22-24') | (df.Q2 == '25-29'), "below 30", "above 30"))
df['below_30'] = df['below_30'].astype(str)

In [5]:
#sort the dataframe by salary list
salarysorter = ['$0-999', '1,000-1,999', '2,000-2,999', '3,000-3,999', '4,000-4,999', '5,000-7,499', '7,500-9,999', '10,000-14,999', '15,000-19,999', '20,000-24,999', '25,000-29,999', '30,000-39,999', '40,000-49,999', '50,000-59,999', '60,000-69,999', '70,000-79,999', '80,000-89,999', '90,000-99,999', '100,000-124,999', '125,000-149,999', '150,000-199,999', '200,000-249,999', '250,000-299,999', '300,000-499,999', '$500,000-999,999', '>$1,000,000', 'NaN']

# Create the dictionary that defines the order for sorting
sorterIndex = dict(zip(salarysorter, range(len(salarysorter))))

# Generate a rank column that will be used to sort
# the dataframe numerically
df['Salary_Rank'] = df['Q29'].map(sorterIndex)

#sorting is done together with the titles (Q23)

In [6]:
df.Q23.unique()

array([nan, 'Data Scientist', 'Software Engineer', 'Research Scientist',
       'Other', 'Developer Advocate',
       'Data Analyst (Business, Marketing, Financial, Quantitative, etc)',
       'Data Engineer', 'Currently not employed',
       'Machine Learning/ MLops Engineer', 'Engineer (non-software)',
       'Teacher / professor', 'Statistician',
       'Manager (Program, Project, Operations, Executive-level, etc)',
       'Data Administrator', 'Data Architect'], dtype=object)

In [10]:
#sort the dataframe by title
titlesorter = ['Data Scientist', 'Data Analyst', 'Software Engineer', 'Teacher / professor', 'Manager', 'Other', 'Research Scientist', 'ML / MLops Engineer', 'Engineer (non-software)', 'Data Engineer', 'Statistician', 'Data Architect', 'Data Administrator', 'Developer Advocate', 'Currently not employed',]

#replace long titles with shorter versions
df = df.replace("Data Analyst (Business, Marketing, Financial, Quantitative, etc)", "Data Analyst")
df = df.replace("Manager (Program, Project, Operations, Executive-level, etc)", "Manager")
df = df.replace("Machine Learning/ MLops Engineer", "ML / MLops Engineer")

# Create the dictionary that defines the order for sorting
sorterIndex2 = dict(zip(titlesorter, range(len(titlesorter))))

# Generate a rank column that will be used to sort
# the dataframe numerically
df['Title_Rank'] = df['Q23'].map(sorterIndex2)

df.sort_values(['Salary_Rank','Title_Rank'], inplace = True)
df.head()

data.add_data_frame(df)

In [8]:
style = Style(
    {
        "legend": {"width":"14em" },
        'fontSize' : '140%',
        'plot' : {'marker' :{ 'label' :{ 'fontSize' : '150%'}}},
    }
)

In [11]:
story = Story(data=data, style=style)
story.set_size("100%", "400px")
story.set_feature("tooltip", True)

#Add event to avoid small values written on the label scale as they overlap
label_handler_method = "if(event.data.text < 120) event.preventDefault()"
#story.add_event("plot-marker-label-draw", label_handler_method)

slide0 = Slide(
#Show number of respondents
    Step(
        Config.bar({
            "x": "Count",
            "title": "No. of respondents"
        })
    )
)
story.add_slide(slide0)

slide1 = Slide()
#Show number of respondents below and above 30
slide1.add_step(
    Step(
        Config.bar({
            "x": ["Count","below_30"]
        })
))

slide1.add_step(
    Step(
        Config.bar({
            "x": "Count",
            "y": "below_30",
            "title": "No. of respondents below and above 30 years"
        })
    )
)
story.add_slide(slide1)

#Add gender
slide2 = Slide()

slide2.add_step(
    Step(
        Config({
            "x": ["Count","Q3"],
            "y": "below_30",
        })
    )
)

slide2.add_step(
    Step(
        Config({
            "color":"Q3",
        }),
    )
)

story.add_slide(slide2)
    

#Show ratio of genders
slide3 = Slide(
    Step(
        Config({
            'align':'stretch'
        })
    )
)
story.add_slide(slide3)

In [12]:
#Remove gender
slide4 = Slide()

slide4.add_step(
    Step(
        Config({
            "align": "none",
        })
    )
)

slide4.add_step(
    Step(
        Config({
            "color":"Q3",
            "x":"Count",
            "color":None
        }),
    )
)

story.add_slide(slide4)

#Add student status
slide5 = Slide()

slide5.add_step(
    Step(
        Config.stackedBar({
            "x": "Count",
            "y": "below_30",
            "stackedBy":"Q5",
            "title": "Are you currently a student?"
        })
    )
)

story.add_slide(slide5)

#Zoom to those who are not students anymore
slide6 = Slide()

slide6.add_step(
    Step(
        Data.filter("record.Q5 === 'No'"),
        Config({
            "title": "Zoom to employees"
        })
    )
)

story.add_slide(slide6)

In [13]:
#Check salary distribution

slide7 = Slide(
    Step(
        Data.filter("record.Q5 === 'No' && record.Q29 !== ''" ),
        Config({
            "title": "Filter those who did not provide salary info"
        })
    )
)
story.add_slide(slide7)

slide8 = Slide()
slide8.add_step(
    Step(
        Config({
            "color": "below_30",
            "x": ["Count","Q29"],
            "legend": "color",
            "label":None
        }),
        Style({
            "plot": {
                "marker": {
                    "colorPalette": "#26EC87FF #29B9BFFF"
                    },
                'xAxis' :{ 'label' :{ 'angle' : '-3.5', 'fontSize' : '0.7em'}
                    }
            }
        })
    )
)

slide8.add_step(
    Step(
        Config.splittedColumn({
            "y":"Count",
            "x":"Q29",
            "splittedBy":"below_30",
            "title":"Salary Distribution"
        })
    )
)

story.add_slide(slide8)

#switch to percentages within below_30 - above 30

In [14]:
slide11 = Slide()

slide11.add_step(
    Step(
        Config({
            "x": ["Count","Q29"],
            "y": "below_30",
            "title": "Let's put back those who didn't provide a salary",
            "split": False
        })
    )
)

slide11.add_step(
    Step(
        Config({
            "label": "Count",
            "x": "Count",
        })
    )
)

slide11.add_step(
    Step(
        Data.filter("record.Q5 === 'No'"),
        Config({
            "title": "And filter the ones that did not add a title"
        })
    )
)

slide11.add_step(
    Step(
    Data.filter("record.Q5 === 'No' && record.Q23 !== ''" ),

))

story.add_slide(slide11)

slide12 = Slide()

slide12.add_step(
    Step(
        Config({
            "x": ["Count","Q23"],
        })
    )
)

slide12.add_step(
    Step(
        Config({
            "x": "Q23",
            "y": ["below_30","Count"]
        })
    )
)

story.add_slide(slide12)

slide13 = Slide()

slide13.add_step(
    Step(
    Config({
        "align":"stretch"
    })
))

story.add_slide(slide13)

# Play the created story - should be at the end of the cell
story.play()

In [ ]:
# If you want to save the story as an interactive HTML
# (containing only the output of the previous cell),
# use the following command:
#story.export_to_html(filename="kaggle.html")

### Learn more about [ipyvizzu-story](https://github.com/vizzuhq/ipyvizzu-story/)